# Data Preprocessing for Spotify Emotion Classification

Data from Kaggle ([link](https://www.kaggle.com/datasets/abdullahorzan/moodify-dataset))

Machine learning engineers are looking to classify songs into one of four categories:
Sad (0), happy (1), energetic (2), and calm (3). For each song, the following features have been collected.

Acousticness: A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.

Danceability: Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.

Duration: Duration of the song in milliseconds. This feature has been log-transformed to make the distribution more reasonable.

Energy: Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.

Instrumentalness: Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.

Liveness: Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides a strong likelihood that the track is live.

Loudness: the overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing the relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typically range between -60 and 0 db.

Speechiness: Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audiobook, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.

Valence: A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).

Tempo: The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, the tempo is the speed or pace of a given piece and derives directly from the average beat duration.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [2]:
# Load the data
with open("../data/data_preprocessing_activity.pkl", "rb") as f:
    data = pickle.load(f)

# For convenience, assume `df` is training data
df = data["df_train"]

# You will not be modifying df_test in this activity
df_test = data["df_test"]

# Data pre-processing activity
In this activity, you will be applying the pre-processing techniques learned in the code-along portion of the workshop to prepare a dataset for a ML model.

The training dataset has been intentionally modifed to simulate issues found in real world datasets. Your task is to apply the relevant pre-processing techniques to the training dataset. The test dataset can be assumed to be a model of appropriate processing applied, and analyzing it may help inform how to proceed.

Both datasets are provided as pandas dataframes, and you will only need to apply operations to the training dataframe to complete the activity. The ML portion under the Testing heading does not need to be touched.

The script will not run successfully until you:
1. Check for and remove unnecessary features
2. Convert the labels from categorical to numeric
3. Drop or interpolate missing values

To improve the performance of the model on the test dataset, we also recommend that you:
1. Check for and remove outliers
2. Scale the data to the same statistics as the test dataset

In [3]:
# Show training data
df.head()

,acousticness,danceability,energy,instrumentalness,liveness,log_duration,log_spec_rate,loudness,random_feature_1,random_feature_2,speechiness,tempo,valence,labels
0,0.0899,0.839,0.726,0.000000,0.110,12.147866,-14.173812,-6.302,0.556511,0.372429,0.1130,140.152,0.158,happy
1,0.0186,0.334,0.430,0.005550,0.129,12.456053,-15.384500,-6.540,0.264990,0.245851,0.0278,100.436,0.257,sad
2,0.9830,0.567,0.125,0.866000,0.111,11.634240,-14.321168,-16.686,0.591935,0.336482,0.0568,76.442,0.319,calm
3,0.2810,0.374,0.534,0.000036,0.110,12.036813,-14.761209,-8.895,0.375105,0.273208,0.0487,149.521,0.345,happy
4,0.1300,0.619,0.972,0.000000,0.782,12.623904,-13.430504,-0.517,0.422110,0.463413,0.4160,169.040,0.584,happy


In [4]:
# Show testing data
df_test.head()

,acousticness,danceability,energy,instrumentalness,liveness,log_duration,log_spec_rate,loudness,speechiness,tempo,valence,labels
0,1.688379,-0.969182,-2.015560,1.831892,-0.720888,-1.224523,0.023370,-4.330251,-0.417876,-1.670009,-0.768619,3
1,1.201873,-0.749425,-1.114305,-0.655007,3.305857,1.990406,0.486308,-1.693653,1.217423,-0.774661,-0.502506,0
2,-1.056486,-0.727986,1.280771,-0.165083,-0.715990,-0.506066,0.121464,0.275515,-0.251574,0.312147,0.162778,2
3,-0.431793,0.333279,0.490829,-0.655007,0.250747,0.120468,-0.879490,0.551495,-0.532208,0.692430,0.645583,1
4,0.859107,-0.288472,0.656752,-0.654435,-0.775990,-0.511439,-0.050580,0.762621,-0.327796,2.367169,1.386899,1


In [5]:
# Remove unnecessary features
df = df.drop(["random_feature_1", "random_feature_2"], axis=1)

In [6]:
# Convert categorical
from sklearn.preprocessing import LabelEncoder

label_mapping = {
    "sad": 0, 
    "happy": 1,
    "energetic": 2,
    "calm": 3
}

df["labels"] = df["labels"].apply(lambda x: label_mapping[x])

In [7]:
# Drop missing rows
df = df.dropna(axis=0)

In [8]:
# Remove outliers heuristically
outlier_potential_columns = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'speechiness', 'valence']

outlier_inds = []
for col in outlier_potential_columns:
    mask = (df[col] < 0) | (df[col] > 1)
    outlier_inds.extend(df[mask].index.tolist())
df = df.drop(outlier_inds, axis=0)

In [9]:
# Scale the data
from sklearn.preprocessing import StandardScaler
df_X = df.drop("labels", axis=1)

scaler = StandardScaler().set_output(transform="pandas")
df_X = scaler.fit_transform(df_X)

In [10]:
df_X

,acousticness,danceability,energy,instrumentalness,liveness,log_duration,log_spec_rate,loudness,speechiness,tempo,valence
0,-0.806225,1.505294,0.603076,-0.680683,-0.481979,-0.309913,0.764883,0.602841,0.217884,0.693994,-1.084255
1,-1.002559,-1.149938,-0.458070,-0.665851,-0.364053,0.452759,-0.985549,0.567258,-0.543555,-0.614414,-0.716205
2,1.653049,0.075149,-1.551480,1.633637,-0.475773,-1.580986,0.551833,-0.949644,-0.284380,-1.404874,-0.485708
3,-0.280005,-0.939622,-0.085235,-0.680587,-0.481979,-0.584735,-0.084385,0.215168,-0.356770,1.002647,-0.389048
4,-0.695804,0.348559,1.484974,-0.680683,3.688902,0.868142,1.839569,1.467742,2.925821,1.645682,0.499478
...,...,...,...,...,...,...,...,...,...,...,...
9995,-1.044360,-0.555797,1.251952,-0.673467,1.758628,0.308855,-0.727240,0.988720,-0.469378,0.331477,0.187193
9996,1.245510,1.042600,0.255336,-0.680683,1.628288,-0.665032,-0.341533,0.453782,-0.463122,-0.192664,1.904763
9997,-0.878370,1.163531,-0.475994,-0.680600,-0.711626,0.708057,1.398826,-0.360435,1.710378,-1.693563,0.339618
9998,1.474062,-1.833462,-1.020907,1.879500,-0.270953,0.160557,-0.568364,-1.337915,-0.429161,0.998726,-0.530320


# Test your model
Run the cells below to test if the data pre-processing applied was appropriate.

In [11]:
# Retrieve training data
X_train = df_X.values
y_train = df["labels"]

In [12]:
from sklearn.linear_model import LogisticRegression

# Instantiate model
# clf = DecisionTreeClassifier(max_depth=5)
clf = LogisticRegression(max_iter=500)

# Retrieve test data
X_test = df_test.drop(["labels"], axis=1).values
y_test = df_test["labels"].values

# Fit the model
clf.fit(X_train, y_train)

# Evaluate performance
train_acc = clf.score(X_train, y_train)
test_acc = clf.score(X_test, y_test)

print(f"Training accuracy: {train_acc*100:.2f}%")
print(f"Testing accuracy:  {test_acc*100:.2f}%")

Training accuracy: 84.52%
Testing accuracy:  83.70%


`LogisticRegression(max_iter=500)`

Train accuracy: 84.52%

Test accuracies
- With all preprocessing: 83.70%
- No outlier drop, no scaling: 65.5% (and max iter)
- No scaling: 70.10% (and reached max iter)
- No outlier drop: 83.50%
- No NaN drop: Error
- No label encoding: 0%
- No removing features: Error